# Edward Leardi

# Metis Project #2

# Using Linear Regression with Ratemyprofessors.com Data

# Part #2: Beautiful Soup HTML Parsing

&nbsp;

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "http://cdn1.theodysseyonline.com/files/2016/02/23/6359184783982034321887056811_RateMyProfessors-logo.png")

## Notes:

* Imports csv file generate by Selenium
* Reads in urls like a dictionary (key=school, value=list of urls to professors at that school)
* My data was split into two csv's, so have to run at minimum twice on each csv
* When originally run, I would slice the csv and define variables like prof_urls_60, to run on the first 60 schools, then run th next 60 by slicing and creating prof_urls_120

In [21]:
from __future__ import print_function, division
import requests
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
import numpy as np
import csv
import time

In [22]:
prof_urls = {}

# Run on both dictionaries
csv_dictionary1 = '/data/my_dictionary_1090.csv'
csv_dictionary2 = '/data/my_dictionary_2000.csv'

with open(csv_dictionary1) as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        school_id = int(row[0])
        prof_url = row[1]
        if school_id not in prof_urls:
            prof_urls[school_id] = [prof_url]
        else:
            prof_urls[school_id].append(prof_url)

# To run in increments, use following format:
prof_urls_60 = dict(list(prof_urls.items())[0:60])




In [23]:
full_list_of_urls = []
full_list_of_prof_lists = []
prof_rating_header_list = ["ID",
                           "URL",
                           "Full_Name",
                           "School",
                           "Num_of_Ratings",
                           "Overall_Rating",
                           "Take_Again",
                           "Difficulty",
                           "Hotness",
                           "TOUGH_GRADER",
                           "GIVES_GOOD_FEEDBACK",
                           "RESPECTED",
                           "GET_READY_TO_READ",
                           "PARTICIPATION_MATTERS",
                           "SKIP_CLASS_YOU_WONT_PASS",
                           "LOTS_OF_HOMEWORK",
                           "INSPIRATIONAL",
                           "BEWARE_OF_POP_QUIZZES",
                           "ACCESSIBLE_OUTSIDE_CLASS",
                           "SO_MANY_PAPERS",
                           "CLEAR_GRADING_CRITERIA",
                           "HILARIOUS",
                           "TEST_HEAVY",
                           "GRADED_BY_FEW_THINGS",
                           "AMAZING_LECTURES",
                           "CARING",
                           "EXTRA_CREDIT",
                           "GROUP_PROJECTS",
                           "LECTURE_HEAVY"
                          ]


In [ ]:
# Run only once
full_list_of_prof_lists.append(prof_rating_header_list)

In [31]:
# Run only once
csv_counter = 0
running_len_of_full_list_of_urls = 0

# If BS gets stuck, or running in increments, manually reset csv_counter & running_len_of_full_list_of_urls
# csv_counter = stuck_count
# running_len_of_full_list_of_urls = stuck_count


In [32]:
# prof_urls_list = either full list or slice (below is the sliced list from cell 4)
prof_urls_list = prof_urls_60

for schoolid in prof_urls_list:
    csv_counter += 1
    for prof_list_url in prof_urls_list[schoolid]:
        if prof_list_url.find("ShowRatings") != -1:
            full_list_of_urls.append(prof_list_url)
            
            
    if csv_counter % 20 == 0:          
        
        # Extract out ratings for each prof url using Beautiful Soup            
        for idx, url in enumerate(full_list_of_urls):
            time.sleep(.25)
            prof_rating_list = []
            response = requests.get(url)
            page = response.text
            soup = BeautifulSoup(page, "lxml")

            prof_rating_list.append(idx + running_len_of_full_list_of_urls)
            prof_rating_list.append(url)

            # First name, last name extraction
            try:
                fname = soup.find_all(class_='pfname')[0].text.strip()
                lname = soup.find_all(class_='plname')[0].text.strip()
                prof_rating_list.append(fname + " " + lname)
            except:
                prof_rating_list.append("error")

            # School extraction
            try:
                school = soup.find_all(class_='school')[0].text.strip()
                prof_rating_list.append(school)
            except:
                prof_rating_list.append("error")

            # Number of ratings extraction
            try:
                num_of_ratings_string = soup.find_all(class_='table-toggle rating-count active')[0].text.strip()
                num_of_ratings = int(re.sub("[^0-9]", "", num_of_ratings_string))
                prof_rating_list.append(num_of_ratings)
            except:
                prof_rating_list.append("error")
                
            # Overall rating extraction
            try:
                overall_rating = float(soup.find_all(class_='grade')[0].text.strip())
                prof_rating_list.append(overall_rating)
            except:
                prof_rating_list.append("error")

            # Would take again extraction
            try:
                would_take_again = soup.find_all(class_='grade')[1].text.strip()
                prof_rating_list.append(would_take_again)
            except:
                prof_rating_list.append("error")

            # Level of diff extraction
            try:
                lvl_of_diff = float(soup.find_all(class_='grade')[2].text.strip())
                prof_rating_list.append(lvl_of_diff)
            except:
                prof_rating_list.append("error")

            # Hotness extraction
            try:
                hotness = 3
                chili_status = soup.find_all('img')[1].prettify()
                if chili_status.find("cold-chili") != -1:
                    hotness = 0
                elif chili_status.find("new-hot-chili") != -1:
                    hotness = 1
                else:
                    hotness = np.nan
                prof_rating_list.append(hotness)
            except:
                prof_rating_list.append("error")

            # All tags extraction
            try:
                tag_box = soup.find_all(class_='tag-box')[0]
                TOUGH = GIVES = RESPECTED = GET = PARTICIPATION = SKIP = LOTS = INSPIRATIONAL = BEWARE = ACCESSIBLE = SO = CLEAR = HILARIOUS = TEST = GRADED = AMAZING = CARING = EXTRA = GROUP = LECTURE = 0
                for x in tag_box.find_all(class_='tag-box-choosetags'):
                    tags = x.text.strip()
                    tags_split = tags.split()
                    tags_first_word = tags_split[0].upper()
                    tag_count = int(re.sub("[^0-9]", "", tags))

                    if tags_first_word == "TOUGH":
                        TOUGH = tag_count
                    elif tags_first_word == "GIVES":
                        GIVES = tag_count
                    elif tags_first_word == "RESPECTED":
                        RESPECTED = tag_count
                    elif tags_first_word == "GET":
                        GET = tag_count
                    elif tags_first_word == "PARTICIPATION":
                        PARTICIPATION = tag_count
                    elif tags_first_word == "SKIP":
                        SKIP = tag_count
                    elif tags_first_word == "LOTS":
                        LOTS = tag_count
                    elif tags_first_word == "INSPIRATIONAL":
                        INSPIRATIONAL = tag_count
                    elif tags_first_word == "BEWARE":
                        BEWARE = tag_count
                    elif tags_first_word == "ACCESSIBLE":
                        ACCESSIBLE = tag_count
                    elif tags_first_word == "SO":
                        SO = tag_count
                    elif tags_first_word == "CLEAR":
                        CLEAR = tag_count
                    elif tags_first_word == "HILARIOUS":
                        HILARIOUS = tag_count
                    elif tags_first_word == "TEST":
                        TEST = tag_count
                    elif tags_first_word == "GRADED":
                        GRADED = tag_count
                    elif tags_first_word == "AMAZING":
                        AMAZING = tag_count
                    elif tags_first_word == "CARING":
                        CARING = tag_count
                    elif tags_first_word == "EXTRA":
                        EXTRA = tag_count
                    elif tags_first_word == "GROUP":
                        GROUP = tag_count
                    elif tags_first_word == "LECTURE":
                        LECTURE = tag_count

                prof_rating_list.append(TOUGH)
                prof_rating_list.append(GIVES)
                prof_rating_list.append(RESPECTED)
                prof_rating_list.append(GET)
                prof_rating_list.append(PARTICIPATION)
                prof_rating_list.append(SKIP)
                prof_rating_list.append(LOTS)
                prof_rating_list.append(INSPIRATIONAL)
                prof_rating_list.append(BEWARE)
                prof_rating_list.append(ACCESSIBLE)
                prof_rating_list.append(SO)
                prof_rating_list.append(CLEAR)
                prof_rating_list.append(HILARIOUS)
                prof_rating_list.append(TEST)
                prof_rating_list.append(GRADED)
                prof_rating_list.append(AMAZING)
                prof_rating_list.append(CARING)
                prof_rating_list.append(EXTRA)
                prof_rating_list.append(GROUP)
                prof_rating_list.append(LECTURE)
            except:
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")
                prof_rating_list.append("error")

            full_list_of_prof_lists.append(prof_rating_list)
            
        
        running_len_of_full_list_of_urls += len(full_list_of_urls)
        
        output_csv_name = "output" + str(csv_counter) + ".csv"
        with open(output_csv_name, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerows(full_list_of_prof_lists)
        full_list_of_urls = []
        full_list_of_prof_lists = []
        
        print("csv counter: " + str(csv_counter))
        print("running len of full list of urls: " + str(running_len_of_full_list_of_urls))


        
for i in range(5):
    os.system("printf '\a'") # or '\7'  

csv counter: 1140
running len of full list of urls: 48512
csv counter: 1160
running len of full list of urls: 49371
csv counter: 1180
running len of full list of urls: 50171
csv counter: 1200
running len of full list of urls: 51098
csv counter: 1220
running len of full list of urls: 52070
csv counter: 1240
running len of full list of urls: 52853
csv counter: 1260
running len of full list of urls: 53665
csv counter: 1280
running len of full list of urls: 54380
csv counter: 1300
running len of full list of urls: 55356
csv counter: 1320
running len of full list of urls: 56522
csv counter: 1340
running len of full list of urls: 58021
csv counter: 1360
running len of full list of urls: 58879
csv counter: 1380
running len of full list of urls: 59722
csv counter: 1400
running len of full list of urls: 60838
csv counter: 1420
running len of full list of urls: 61696
csv counter: 1440
running len of full list of urls: 62439
csv counter: 1460
running len of full list of urls: 63866
csv counter: 1

KeyboardInterrupt: 

In [20]:
# Run to check progress

print(csv_counter)
print(running_len_of_full_list_of_urls)


1020
40775
